# Import

Libraries

In [72]:
import os
import shutil
import tensorflow as tf
import pandas as pd
import numpy as np

Path

In [ ]:
dir_dataset = r"C:\Users\USER\Desktop\Projects\PASD_Deteksi_Penyakit_Tanaman\Dataset\raw\color"

dir_train = r"C:\Users\USER\Desktop\Projects\PASD_Deteksi_Penyakit_Tanaman\Dataset\train"
dir_val = r"C:\Users\USER\Desktop\Projects\PASD_Deteksi_Penyakit_Tanaman\Dataset\val"
dir_test = r"C:\Users\USER\Desktop\Projects\PASD_Deteksi_Penyakit_Tanaman\Dataset\test"

df_train = pd.read_excel(r"C:\Users\USER\Desktop\Projects\PASD_Deteksi_Penyakit_Tanaman\Dataset\split\train.xlsx")
df_val = pd.read_excel(r"C:\Users\USER\Desktop\Projects\PASD_Deteksi_Penyakit_Tanaman\Dataset\split\val.xlsx")
df_test = pd.read_excel(r"C:\Users\USER\Desktop\Projects\PASD_Deteksi_Penyakit_Tanaman\Dataset\split\test.xlsx")

# Preprocessing

Create Folders

In [ ]:
for folder in ["train", "val", "test"]:
    if folder not in os.listdir(r"C:\Users\USER\Desktop\Projects\PASD_Deteksi_Penyakit_Tanaman\Dataset"):
        os.mkdir(os.path.join(r"C:\Users\USER\Desktop\Projects\PASD_Deteksi_Penyakit_Tanaman\Dataset", folder))

Move val data

In [65]:
for label in df_val["label"].unique():
    if label not in os.listdir(dir_val):
        os.mkdir(os.path.join(dir_val, label))
        
    files = df_val[df_val["label"] == label]["file_name"]
    for file in files:
        dir_image_in = os.path.join(dir_dataset, label, file)
        dir_image_out = os.path.join(dir_val, label, file)
        shutil.copyfile(dir_image_in, dir_image_out)

In [67]:
total = 0
for folder in os.listdir(dir_val):
    total += len(os.listdir(os.path.join(dir_val, folder)))
print(total, len(df_val))

1816 1816


Move test data

In [68]:
for label in df_test["label"].unique():
    if label not in os.listdir(dir_test):
        os.mkdir(os.path.join(dir_test, label))
        
    files = df_test[df_test["label"] == label]["file_name"]
    for file in files:
        dir_image_in = os.path.join(dir_dataset, label, file)
        dir_image_out = os.path.join(dir_test, label, file)
        shutil.copyfile(dir_image_in, dir_image_out)

In [69]:
total = 0
for folder in os.listdir(dir_test):
    total += len(os.listdir(os.path.join(dir_test, folder)))
print(total, len(df_test))

1816 1816


Functions

In [74]:
def augment_image(image, file_name, output_dir):
    augmented_images = []
    augmented_images.append((image, "original"))
    augmented_images.append((tf.image.flip_left_right(image), "flip_lr"))
    augmented_images.append((tf.image.rot90(image), "rot90"))
    scaled = tf.image.resize_with_crop_or_pad(image, 200, 200)
    augmented_images.append((tf.image.resize(scaled, [256, 256]), "scaled"))
    augmented_images.append((tf.image.random_brightness(image, max_delta=0.2), "brightness"))
    
    for img, augmentation_type in augmented_images:
        augmented_file_name = f"{os.path.splitext(file_name)[0]}_{augmentation_type}.jpg"
        augmented_file_path = os.path.join(output_dir, augmented_file_name)
        tf.keras.preprocessing.image.save_img(augmented_file_path, img)

Augment and Move train data

In [75]:
for label in df_train["label"].unique():
    if label not in os.listdir(dir_train):
        os.mkdir(os.path.join(dir_train, label))
        
    files = df_train[df_train["label"] == label]["file_name"]
    for file in files:
        dir_image_in = os.path.join(dir_dataset, label, file)
        dir_image_out = os.path.join(dir_train, label, file)
        shutil.copyfile(dir_image_in, dir_image_out)
        
        image = tf.keras.preprocessing.image.load_img(dir_image_in)
        image = tf.keras.preprocessing.image.img_to_array(image)
        
        augment_image(image, file, os.path.join(dir_train, label))

In [76]:
total = 0
for folder in os.listdir(dir_train):
    total += len(os.listdir(os.path.join(dir_train, folder)))
print(total, len(df_train))

87168 14528
